To do's:
- streamline the pre/post processing with pipelines
- clean up things, e.g. throw away train signal labels earlier for more trust
- allow wrapped models to continue training for an extra n epochs whenever fit is called
- maybe ensembling?
- documentation

Sanity checks:
- epoch number matching between checkpoint name and position in loss array?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from os.path import join
from sklearn.metrics import roc_curve
from sklearn.neighbors import KernelDensity

from conditional_normalizing_flow import ConditionalNormalizingFlow
from neural_network_classifier import NeuralNetworkClassifier

In [ ]:
# :sunglasses:
plt.style.use('dark_background')

In [ ]:
# data loading
data_path = "./input_data/"
outerdata_train = np.load(join(data_path, "outerdata_train.npy"))
outerdata_val = np.load(join(data_path, "outerdata_val.npy"))
innerdata_train = np.load(join(data_path, "innerdata_train.npy"))
innerdata_val = np.load(join(data_path, "innerdata_val.npy"))
innerdata_test = np.load(join(data_path, "innerdata_test.npy"))

In [ ]:
# some data preprocessing functions
def logit_transform(x, min_vals, max_vals):
    with np.errstate(divide='ignore', invalid='ignore'):
        x_norm = (x - min_vals) / (max_vals - min_vals)
        logit = np.log(x_norm / (1 - x_norm))
    domain_mask = ~(np.isnan(logit).any(axis=1) | np.isinf(logit).any(axis=1))
    return logit, domain_mask

def standardize(x, mean, std):
    return (x - mean) / std

def inverse_logit_transform(x, min_vals, max_vals):
    x_norm = 1 / (1 + np.exp(-x))
    return x_norm * (max_vals - min_vals) + min_vals

def inverse_standardize(x, mean, std):
    return x * std + mean

In [ ]:
# flow data preprocessing

# the logit min/max and standard scaler parameters are derived on the outer train set
outer_x_params = {}
outer_x_params["min"] = np.min(outerdata_train[:, 1:-1], axis=0)
outer_x_params["max"] = np.max(outerdata_train[:, 1:-1], axis=0)

preprocessed_outerdata_train_x, mask = logit_transform(outerdata_train[:, 1:-1],
                                                       outer_x_params["min"],
                                                       outer_x_params["max"])
preprocessed_outerdata_train = np.hstack([outerdata_train[:, 0:1],
                                          preprocessed_outerdata_train_x,
                                          outerdata_train[:, -1:]])[mask]

outer_x_params["mean"] = np.mean(preprocessed_outerdata_train[:, 1:-1], axis=0)
outer_x_params["std"] = np.std(preprocessed_outerdata_train[:, 1:-1], axis=0)
preprocessed_outerdata_train[:, 1:-1] = standardize(preprocessed_outerdata_train[:, 1:-1],
                                                    outer_x_params["mean"],
                                                    outer_x_params["std"])

# outer validation set if one wants to train a new flow
preprocessed_outerdata_val_x, mask = logit_transform(outerdata_val[:, 1:-1],
                                                     outer_x_params["min"],
                                                     outer_x_params["max"])
preprocessed_outerdata_val = np.hstack([outerdata_val[:, 0:1],
                                        preprocessed_outerdata_val_x,
                                        outerdata_val[:, -1:]])[mask]
preprocessed_outerdata_val[:, 1:-1] = standardize(preprocessed_outerdata_val[:, 1:-1],
                                                  outer_x_params["mean"],
                                                  outer_x_params["std"])

In [ ]:
# either train new flow model from scratch
m_train = preprocessed_outerdata_train[:, 0]
x_train = preprocessed_outerdata_train[:, 1:-1]
m_val = preprocessed_outerdata_val[:, 0]
x_val = preprocessed_outerdata_val[:, 1:-1]

flow_savedir = "./trained_flows_sklearn_new/"
flow_model = ConditionalNormalizingFlow(save_path=flow_savedir)
flow_model.fit(m_train, x_train, m_val, x_val, epochs=50, verbose=True)

# then go back to the optimal epoch checkpoint
flow_model.load_best_model()

In [ ]:
# or loading existing flow model
flow_savedir = "./trained_flows_sklearn/"
flow_model = ConditionalNormalizingFlow(save_path=flow_savedir)
flow_model.load_best_model()

In [ ]:
# fitting a KDE for the mass distribution based on the inner training set

# we also perform a logit first to stretch out the hard boundaries
m_train = innerdata_train[:, 0].reshape(-1, 1)
inner_m_params = {"min": np.min(m_train),
                  "max": np.max(m_train)}
m_train, mask = logit_transform(m_train,
                                inner_m_params["min"],
                                inner_m_params["max"])
m_train = m_train[mask]

kde_model = KernelDensity(bandwidth=0.01, kernel='gaussian')
kde_model.fit(m_train)

# now let's sample 4x the number of training data
m_samples = kde_model.sample(4*len(m_train)).astype(np.float32)
m_samples = inverse_logit_transform(m_samples,
                                    inner_m_params["min"],
                                    inner_m_params["max"])

In [ ]:
# drawing samples from the flow model with the KDE samples as conditional
x_samples = flow_model.sample(m_samples)

x_samples = inverse_standardize(x_samples,
                                outer_x_params["mean"],
                                outer_x_params["std"])
x_samples = inverse_logit_transform(x_samples,
                                    outer_x_params["min"],
                                    outer_x_params["max"])

samples = np.hstack([m_samples, x_samples, np.zeros((m_samples.shape[0], 1))])

In [ ]:
# comparing samples to inner background (idealized sanity check)

for i in range(5):
    _, binning, _ = plt.hist(innerdata_test[innerdata_test[:, -1] == 0, i],
                             bins=100, label="data background",
                             density=True, histtype="step")
    _ = plt.hist(samples[:, i],
                 bins=binning, label="sampled background",
                 density=True, histtype="step")
    plt.legend()
    plt.ylim(0, plt.gca().get_ylim()[1] * 1.2)
    plt.xlabel("feature {}".format(i))
    plt.ylabel("counts")
    plt.show()


In [ ]:
# preprocess inner data for classifier training

inner_x_params = {}
inner_x_params["mean"] = np.mean(innerdata_train[:, 1:-1], axis=0)
inner_x_params["std"] = np.std(innerdata_train[:, 1:-1], axis=0)
preprocessed_innerdata_train_x = standardize(innerdata_train[:, 1:-1],
                                             inner_x_params["mean"],
                                             inner_x_params["std"])
preprocessed_innerdata_train = np.hstack([innerdata_train[:, 0].reshape(-1, 1),
                                          preprocessed_innerdata_train_x,
                                          np.ones((len(innerdata_train), 1))])

# the signal label column is replaced by a 1 for "data"
preprocessed_innerdata_val_x = standardize(innerdata_val[:, 1:-1],
                                           inner_x_params["mean"],
                                           inner_x_params["std"])
preprocessed_innerdata_val = np.hstack([innerdata_val[:, 0].reshape(-1, 1),
                                        preprocessed_innerdata_val_x,
                                        np.ones((len(innerdata_val), 1))])

preprocessed_innerdata_test_x = standardize(innerdata_test[:, 1:-1],
                                            inner_x_params["mean"],
                                            inner_x_params["std"])
preprocessed_innerdata_test = np.hstack([innerdata_test[:, 0].reshape(-1, 1),
                                         preprocessed_innerdata_test_x,
                                         innerdata_test[:, -1].reshape(-1, 1)])

# same preprocessing for samples, but the signal label column is set to 0 for "samples"
preprocessed_samples_x = standardize(samples[:, 1:-1],
                                     inner_x_params["mean"],
                                     inner_x_params["std"])
preprocessed_samples = np.hstack([samples[:, 0].reshape(-1, 1),
                                  preprocessed_samples_x,
                                  np.zeros((len(samples), 1))])

# mix data and samples into train/val sets together proportionally
n_train = len(preprocessed_innerdata_train)
n_val = len(preprocessed_innerdata_val)
n_samples_train = int(n_train / (n_train + n_val) * len(preprocessed_samples))
preprocessed_samples_train = preprocessed_samples[:n_samples_train]
preprocessed_samples_val = preprocessed_samples[n_samples_train:]

preprocessed_train_set = np.vstack([preprocessed_innerdata_train,
                                    preprocessed_samples_train])
preprocessed_val_set = np.vstack([preprocessed_innerdata_val,
                                  preprocessed_samples_val])

np.random.shuffle(preprocessed_train_set)
np.random.shuffle(preprocessed_val_set)

# also preprocess the test set for evaluation, but leave the signal labels untouched
preprocessed_innerdata_test_x = standardize(innerdata_test[:, 1:-1],
                                            inner_x_params["mean"],
                                            inner_x_params["std"])
preprocessed_innerdata_test = np.hstack([innerdata_test[:, 0].reshape(-1, 1),
                                         preprocessed_innerdata_test_x,
                                         innerdata_test[:, -1].reshape(-1, 1)])

In [ ]:
# either train new NN classifier to distinguish between real inner data and samples

x_train = preprocessed_train_set[:, 1:-1]
y_train = preprocessed_train_set[:, -1]
x_val = preprocessed_val_set[:, 1:-1]
y_val = preprocessed_val_set[:, -1]

classifier_savedir = "trained_classifiers_sklearn_new/"
classifier_model = NeuralNetworkClassifier(save_path=classifier_savedir,
                                           n_inputs=x_train.shape[1])
classifier_model.fit(x_train, y_train, x_val, y_val,
                     epochs=100, verbose=True)

# then go back to the optimal epoch checkpoint
classifier_model.load_best_model()

In [ ]:
# or loading existing classifer model

classifier_savedir = "trained_classifiers_sklearn/"
classifier_model = NeuralNetworkClassifier(save_path=classifier_savedir,
                                           n_inputs=x_train.shape[1])
classifier_model.load_best_model()

In [ ]:
# now let's evaluate the signal extraction performance

x_test = preprocessed_innerdata_test[:, 1:-1]
y_test = preprocessed_innerdata_test[:, -1]

preds_test = classifier_model.predict(x_test)

with np.errstate(divide='ignore', invalid='ignore'):
    fpr, tpr, _ = roc_curve(y_test, preds_test)
    sic = tpr / np.sqrt(fpr)

    random_tpr = np.linspace(0, 1, 300)
    random_sic = random_tpr / np.sqrt(random_tpr)

plt.plot(tpr, sic, label="CATHODE")
plt.plot(random_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()